<a href="https://colab.research.google.com/github/spendnetwork/charts/blob/master/number_of_buyers_deloitte_chart_opensourced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import altair as alt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [0]:
url = 'https://raw.githubusercontent.com/spendnetwork/charts/master/number_of_buyers_deloitte.csv'
df = pd.read_csv(url)
#print(df)

In [0]:
df['releasedate']=pd.to_datetime(df['releasedate']).dt.strftime('%m/%d/%Y')

In [0]:
df1=(df.groupby('releasedate').agg({'buyer': pd.Series.nunique}))
#df1.head()

In [16]:
#df1.columns = df1.columns.map('_'.join)
df1 = df1.reset_index()
df1.head()

,releasedate,buyer
0,01/02/2019,1
1,01/04/2018,1
2,01/04/2019,1
3,01/06/2017,2
4,01/08/2019,1


In [0]:
#df1['releasedate']=pd.to_datetime(df1['releasedate'].dt.strftime('%m/%d/%Y'))
#df1['releasedate']=pd.to_datetime(df1['releasedate']).dt.strftime('%m/%d/%Y') 
#df1.head()

In [0]:
def which_caret(stat_value):
        stat_value = int(stat_value)
        ## returning html that deploys Font Awesome icons, can be changed according to icon set in use
        if stat_value > 0:
            caret = u"\u25B2"
        elif stat_value < 0:
            caret = u"\u25BC"
        else:
            caret = u"\u003D"
        return caret

def rolling_twelve_month_trend(df, colname, agg_type):        
        df2=df1['releasedate'] = pd.to_datetime(df1['releasedate'])
        p2mask = (df1['releasedate'] > datetime.today() - relativedelta(years=1)) & (df1['releasedate'] <= datetime.today())
        p1mask = (df1['releasedate'] > datetime.today() - relativedelta(years=2)) & (
                    df1['releasedate'] <= datetime.today() - relativedelta(years=1))
        p2 = df.loc[p2mask]
        p1 = df.loc[p1mask]

        perc = None
        if agg_type == 'sum':
            perc = float(p2[colname].sum() - p1[colname].sum()) / abs(p1[colname].sum()) * 100
        elif agg_type == 'mean':
            perc = float(p2[colname].mean() - p1[colname].mean()) / abs(p1[colname].mean()) * 100
        return perc
  
def inc_dec(stat_value):
        stat_value = int(stat_value)
        ## returning html that deploys Font Awesome icons, can be changed according to icon set in use
        if stat_value > 0:
            change = 'increase'
        elif stat_value < 0:
            change = 'decrease'
        else:
            change = 'no change'
        return change

In [0]:
perc = rolling_twelve_month_trend(df1, 'buyer', 'sum')
caret = which_caret(perc)
change = inc_dec(perc)

In [20]:
perc, caret

(45.0, '▲')

In [0]:
  nearest = alt.selection(type='single', nearest=True, on='mouseover',fields=['releasedate'], empty='none')

In [22]:
        line=alt.Chart(df1, width=1200, height=250, title="Count of buyers of Deloitte LLP").mark_line(color='#91D739').encode(
            x=alt.X('yearmonth(releasedate):O',
                    axis=alt.Axis(tickCount=12, title="https://openopps.com", 
                grid=False, 
                labelFont='Helvetica', 
                labelFontSize=12, 
                labelAngle=0, 
                titleAlign='left',
                titleAnchor='start',
                titleFont='Helvetica',
                titleFontSize= 11,
                titleFontWeight= 300,
                titleX= -0,
                titleY= 30) 
                   ),
            y=alt.Y('sum(buyer):Q',
                     axis=alt.Axis(title=(caret +' '+ str(int(perc))+'% ' + change + ' this year in comparison with the previous year.' ),
                                    format=".0s",
                                    titleAlign= 'left',
                                    titleAnchor='start',
                                    titleAngle= 0,
                                    titleFont= 'Helvetica',
                                    titleFontSize= 12,
                                    titleFontWeight= 400,
                                    titleX= -0,
                                    titleY= -30,)
                   ),
            )
        selectors = alt.Chart(df1).mark_point().encode(
            x='yearmonth(releasedate):O',
            opacity=alt.value(0),
        ).add_selection(
            nearest
        )

        points = line.mark_point().encode(
            opacity=alt.condition(nearest, alt.value(1), alt.value(0))
        )


        text = line.mark_text(align='left', dx=10, dy=-10).encode(
            text=alt.condition(nearest, 'sum(buyer):Q', alt.value(' '))
        )

        rules = alt.Chart(df1).mark_rule(color='gray').encode(
            x='yearmonth(releasedate):O',
        ).transform_filter(
            nearest
        )

        chart = alt.layer(
            line, selectors, points, rules, text
        ).configure_view(
          strokeWidth=0
        ).configure_title(fontSize=18, anchor='start',
                color= "#91D739", font='Helvetica', fontWeight=400)

        chart

alt.LayerChart(...)